In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## AI 모의 면접봇

1. 지원하고자 하는 회사의 정보 수집(채용공고-조건,우대조건,인재상,...)
2. 지원자의 정보 준비 (인적사항, 학력/경력, 포트폴리오,...)
3. 1과 2에서 수집한 정보를 바탕으로 모의 면접 진행
    - 면접관의 스타일 선택 (성격, 성향, 실무진/임원진 등...)
4. 모의 면접 결과 및 피드백 안내

- 결과 시연 + langsmith history

In [2]:
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ChatMessage,HumanMessage,AIMessage
from langchain_core.documents import Document
import os

### 회사꺼 백터변환

In [3]:
with open('./data/company_info.txt' , 'r', encoding='utf-8') as f:
    company_info = f.read()
    
# 개행이 두 번 들어간 부분을 기준으로 텍스트 분할
company_info_list = company_info.split('\n\n')

# 개행 띄어쓰기로 변환
company_info_list = [info.replace('\n', ' ') for info in company_info_list]

company_info_list

['1. 이스트소프트 조건: LLM/NLP 관련 분야 석사 학위 이상 또는 그에 준하는 3년 이상의 경력 알고리즘과 머신러닝, LLM 분야에 대한 깊은 이해 Pytorch를 활용한 코드 작성 및 실험 능숙 우대조건: LLM 학습 및 최적화 경험 분산 학습 시스템 구축 경험 GPU 자원 최적화 경험 모델 경량화 및 추론 성능 최적화 경험 AWS, Azure 등 클라우드 플랫폼 사용 경험 필요 기술 스택: Git AI/인공지능 Linux Machine Learning Deep Learning AWS TensorFlow Python Azure PyTorch 출처:  JOBKOREA.CO.KR',
 '2. 업스테이지 조건: LLM 알고리즘/데이터 연구·개발 경험 LLM Post-training 경험 다양한 협업 환경에서의 원활한 의사소통 능력 우대조건: 대규모 모델 학습을 위한 프레임워크 사용 경험 (예: DeepSpeed) LLM 기반 제품 개발 경험 AI 및 NLP 관련 국제 학회 논문 게재 경험 필요 기술 스택: AI/인공지능 NLP TensorFlow PyTorch 출처:  JOBKOREA.CO.KR',
 '3. 삼정KPMG 조건: ML/AI/NL/LLM 분야 경력자 관련 프로젝트 경험 및 성과 보유자 우대조건: 금융 및 컨설팅 업계 경험자 데이터 분석 및 처리 능력 우수자 필요 기술 스택: Machine Learning AI Natural Language Processing LLM 출처:  JOBKOREA.CO.KR',
 '4. 아이브릭스 조건: LLM 서비스 구축 및 운영 경험 AI 모델 개발 및 최적화 능력 우대조건: 대규모 데이터 처리 경험 클라우드 환경에서의 서비스 개발 경험 필요 기술 스택: AI/인공지능 Machine Learning Python TensorFlow 출처:  잡코리아',
 '5. 비전스페이스 조건: 생성형 AI, LLM, RAG R&D 개발 경험 로봇산업 관련 프로젝트 참여 경험 우대조건: 데이터 사이언스 및 분석 능력 우수자 유연

In [4]:
documents = [Document(page_content=info) for info in company_info_list]
documents

[Document(metadata={}, page_content='1. 이스트소프트 조건: LLM/NLP 관련 분야 석사 학위 이상 또는 그에 준하는 3년 이상의 경력 알고리즘과 머신러닝, LLM 분야에 대한 깊은 이해 Pytorch를 활용한 코드 작성 및 실험 능숙 우대조건: LLM 학습 및 최적화 경험 분산 학습 시스템 구축 경험 GPU 자원 최적화 경험 모델 경량화 및 추론 성능 최적화 경험 AWS, Azure 등 클라우드 플랫폼 사용 경험 필요 기술 스택: Git AI/인공지능 Linux Machine Learning Deep Learning AWS TensorFlow Python Azure PyTorch 출처:  JOBKOREA.CO.KR'),
 Document(metadata={}, page_content='2. 업스테이지 조건: LLM 알고리즘/데이터 연구·개발 경험 LLM Post-training 경험 다양한 협업 환경에서의 원활한 의사소통 능력 우대조건: 대규모 모델 학습을 위한 프레임워크 사용 경험 (예: DeepSpeed) LLM 기반 제품 개발 경험 AI 및 NLP 관련 국제 학회 논문 게재 경험 필요 기술 스택: AI/인공지능 NLP TensorFlow PyTorch 출처:  JOBKOREA.CO.KR'),
 Document(metadata={}, page_content='3. 삼정KPMG 조건: ML/AI/NL/LLM 분야 경력자 관련 프로젝트 경험 및 성과 보유자 우대조건: 금융 및 컨설팅 업계 경험자 데이터 분석 및 처리 능력 우수자 필요 기술 스택: Machine Learning AI Natural Language Processing LLM 출처:  JOBKOREA.CO.KR'),
 Document(metadata={}, page_content='4. 아이브릭스 조건: LLM 서비스 구축 및 운영 경험 AI 모델 개발 및 최적화 능력 우대조건: 대규모 데이터 처리 경험 클라우드 환경에서의 서비스 개발 경험 필요 기술 스택: AI/

In [5]:
# 임베딩 모델 생성
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [13]:
from langchain_chroma.vectorstores import Chroma
vector_store_company = Chroma(
    embedding_function=embedding_model,
    persist_directory='./data/company_info_vector_store'
)
vector_store_company.add_documents(documents)

['9dee60f5-6a64-47e0-b082-249cafcc47e3',
 'f8c9192c-c7d7-4e77-b276-0fe00a99ee19',
 '3a396134-6aa6-4549-80a4-6f2ec8644de5',
 'ff9bd2b4-4b2a-4571-9edf-8d5fe64c2a53',
 'ddd57042-b3bd-40d5-b2ac-d8d1f055ade4',
 'fa4db59d-5db7-4452-ac4d-27961e7f5fdb']

### 내 정보 벡터 변환

In [7]:
with open('./data/job_application.txt' , 'r', encoding='utf-8') as f:
    my_info = f.read()

my_info = my_info.replace('\n\n', ' ')
my_info = my_info.replace('\n', ' ')
my_info

'이력서 1. 인적사항 - 이름: 이재혁 - 성별: 남자 - 생년월일: 2000년생 - 병역사항: 현역 만기 전역 2. 학력 - 학사 학위 취득  3. 경력 및 교육 - 풀스택 개발자 수료 과정 이수 - LLM서비스 개발자 수료중  4. 포트폴리오 1) 프로젝트 경험 - 대실 예약 시스템 개발 (Spring Boot, React, MySQL)   - Spring Boot 기반의 백엔드 API 개발   - React를 활용한 프론트엔드 구현   - MySQL을 이용한 데이터베이스 설계 및 최적화   - Docker를 활용한 배포 환경 구성 - CI/CD 자동화 구축 (Jenkins, Docker, GitHub Actions)   - Jenkins와 GitHub Actions를 활용한 배포 자동화   - Docker 컨테이너 기반으로 프론트엔드 및 백엔드 개별 배포   - 코드 변경 시 자동 빌드 및 테스트 실행 - 머신러닝 및 딥러닝 모델 개발 및 배포 (Python, TensorFlow, PyTorch, Streamlit, XGBoost, RandomForest, Transformer, LangChain)   - 기대수명 예측 및 음악 사이트 이탈률 분석 프로젝트 진행   - Streamlit을 활용한 인터랙티브 웹 애플리케이션 구현   - XGBoost 및 랜덤 포레스트를 활용한 회귀 및 분류 모델 개발   - TensorFlow 및 PyTorch를 활용한 딥러닝 모델 구축 및 최적화   - CNN, RNN을 활용한 이미지 및 시계열 데이터 분석 경험   - Transformer 기반 자연어 처리 모델 개발 및 최적화   - LangChain을 활용한 LLM 기반 응용 서비스 개발 및 데이터 파이프라인 구성 2) 보유 기술 - 프론트엔드: React, JavaScript, TypeScript - 백엔드: Spring Boot, Java - 데이터베이스: MySQL, PostgreSQL - 배포 및 운영: Docker, Jenkins, GitHub Action

In [10]:
docs = [Document(page_content=my_info)]
docs

[Document(metadata={}, page_content='이력서 1. 인적사항 - 이름: 이재혁 - 성별: 남자 - 생년월일: 2000년생 - 병역사항: 현역 만기 전역 2. 학력 - 학사 학위 취득  3. 경력 및 교육 - 풀스택 개발자 수료 과정 이수 - LLM서비스 개발자 수료중  4. 포트폴리오 1) 프로젝트 경험 - 대실 예약 시스템 개발 (Spring Boot, React, MySQL)   - Spring Boot 기반의 백엔드 API 개발   - React를 활용한 프론트엔드 구현   - MySQL을 이용한 데이터베이스 설계 및 최적화   - Docker를 활용한 배포 환경 구성 - CI/CD 자동화 구축 (Jenkins, Docker, GitHub Actions)   - Jenkins와 GitHub Actions를 활용한 배포 자동화   - Docker 컨테이너 기반으로 프론트엔드 및 백엔드 개별 배포   - 코드 변경 시 자동 빌드 및 테스트 실행 - 머신러닝 및 딥러닝 모델 개발 및 배포 (Python, TensorFlow, PyTorch, Streamlit, XGBoost, RandomForest, Transformer, LangChain)   - 기대수명 예측 및 음악 사이트 이탈률 분석 프로젝트 진행   - Streamlit을 활용한 인터랙티브 웹 애플리케이션 구현   - XGBoost 및 랜덤 포레스트를 활용한 회귀 및 분류 모델 개발   - TensorFlow 및 PyTorch를 활용한 딥러닝 모델 구축 및 최적화   - CNN, RNN을 활용한 이미지 및 시계열 데이터 분석 경험   - Transformer 기반 자연어 처리 모델 개발 및 최적화   - LangChain을 활용한 LLM 기반 응용 서비스 개발 및 데이터 파이프라인 구성 2) 보유 기술 - 프론트엔드: React, JavaScript, TypeScript - 백엔드: Spring Boot, Java - 데이터베이스: MySQL, PostgreSQL - 배포 

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 각 chunk의 최대 문자 수
    chunk_overlap=20 # 인접한 텍스트 조각 간 겹치는 문자 수 (기본값: 200)
                        # 텍스트 분할 구분자 우선순위 (기본값: ['\n\n', '\n', ' ', ''])
)
docs = splitter.split_documents(docs)
documents.extend(docs)
documents

[Document(metadata={}, page_content='1. 이스트소프트 조건: LLM/NLP 관련 분야 석사 학위 이상 또는 그에 준하는 3년 이상의 경력 알고리즘과 머신러닝, LLM 분야에 대한 깊은 이해 Pytorch를 활용한 코드 작성 및 실험 능숙 우대조건: LLM 학습 및 최적화 경험 분산 학습 시스템 구축 경험 GPU 자원 최적화 경험 모델 경량화 및 추론 성능 최적화 경험 AWS, Azure 등 클라우드 플랫폼 사용 경험 필요 기술 스택: Git AI/인공지능 Linux Machine Learning Deep Learning AWS TensorFlow Python Azure PyTorch 출처:  JOBKOREA.CO.KR'),
 Document(metadata={}, page_content='2. 업스테이지 조건: LLM 알고리즘/데이터 연구·개발 경험 LLM Post-training 경험 다양한 협업 환경에서의 원활한 의사소통 능력 우대조건: 대규모 모델 학습을 위한 프레임워크 사용 경험 (예: DeepSpeed) LLM 기반 제품 개발 경험 AI 및 NLP 관련 국제 학회 논문 게재 경험 필요 기술 스택: AI/인공지능 NLP TensorFlow PyTorch 출처:  JOBKOREA.CO.KR'),
 Document(metadata={}, page_content='3. 삼정KPMG 조건: ML/AI/NL/LLM 분야 경력자 관련 프로젝트 경험 및 성과 보유자 우대조건: 금융 및 컨설팅 업계 경험자 데이터 분석 및 처리 능력 우수자 필요 기술 스택: Machine Learning AI Natural Language Processing LLM 출처:  JOBKOREA.CO.KR'),
 Document(metadata={}, page_content='4. 아이브릭스 조건: LLM 서비스 구축 및 운영 경험 AI 모델 개발 및 최적화 능력 우대조건: 대규모 데이터 처리 경험 클라우드 환경에서의 서비스 개발 경험 필요 기술 스택: AI/

In [35]:
vector_store_company.add_documents(documents)

['6c4650fc-119d-4b30-bb28-bd3bdfdb4b9a',
 'b50c2521-e423-4c8b-b010-c4fbe0d773f5',
 '50e9f4d5-0ce9-4600-bd0b-ca93b6726a1e',
 '1a2c80ec-0331-4d93-a5d7-61ef0c0ce6a0',
 '6f037cdf-490c-469b-bfa2-394489f24620',
 '8a3987e7-f5ba-452e-9253-8d78eb7838f4',
 '9ec71d29-2a60-4254-9735-e04bcbe11753',
 'b00004dc-21cf-4843-af7d-de05d8747858',
 '1b45d51b-ab78-42cb-a3f3-2160dd1587fe',
 '19c0e7e2-77d2-4a23-902b-0642aeb0126d',
 '071b6a9d-718f-4819-b175-ac61e0034f6d',
 '6406a011-08e4-4b6f-a4d9-ff60025ed371',
 '194a15e9-0d97-49d2-8c96-d5b2f03f2177',
 '4bc9c1d8-fc63-4932-a8e6-af21eeb9b6f4',
 '5f8a7d46-1f0a-4b71-b7a0-b2ebe675463e',
 '203298cd-6ffd-418c-ab8d-29107c6909cf',
 'cd552eed-9624-424c-8c17-261f51a8f31e',
 '63680175-0348-487c-8793-f744d2b1fb5b',
 '183a2426-4187-42b4-8304-aab4b6602ebf',
 '0e860ded-2f3b-48be-bba7-06c8ec7dca3f']

### model 생성

In [36]:
from langchain_openai import ChatOpenAI
# llm 설정 (모델 생성)
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature=0.5
)

In [67]:
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate([
    ('system',"""
    너는 면접관이야 
    
    다음 '지원자의 정보'를 바탕으로 '회사 정보'를 참고하여 지원자에게 알맞는 질문을 해서 모의 면접을 진행해줘.
    회사내 필요한 기술스택 혹은 정보에서 담겨진 궁금증 다 물어봐줘
    면접관의 성격은 친절하거나 까칠하거나 너가 편한대로 해도 무방해
    면접관의 직업은 현직자이거나 임원 혹은 프로젝트매니저 아무거나 상관없어
    
    ---
    지원자의 정보 : {context}
    문서 : {context}
    ---
    <면접 흐름>
    면접관이 먼저 질문 후 지원자가 답변을 하게 됩니다.
    면접관 : [정보와 문서를 토대로 나온 질문]
    지원자 : {query}
    이 흐름을 3번 반복해.
    
    <3번 질문 후 면접 결과 및 피드백>
    면접 결과와 피드백을 해줘.
    
    <답변에 대한 조건>
    - 회사의 요구사항에 대해 질문 할 것
    - 지원자의 정보에 대한 질문을 할 것
    """, ),
    MessagesPlaceholder(variable_name='history'),
    ('human','{query}')
])

In [49]:
retriever = vector_store_company.as_retriever()

In [79]:
from langchain_core.messages import HumanMessage, SystemMessage
sys_inst = """
당신은 면접관입니다. 
지원자에게 질문을 여러번 진행하고 질문을 다 했으면 면접 결과와 피드백을 해주세요.
AI, LLM 관련한 질문을 해주세요.
"""
msgs = [SystemMessage(sys_inst)]
res = model.invoke(msgs)
msgs.append(res)
print()
print(res.content)

while True:
    reply = input()
    print(reply)
    msgs.append(HumanMessage(reply))
    if "종료" in reply:
        break
    res = model.invoke(msgs)
    msgs.append(res)
    print(res.content)
    
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""
    다음 '질문'에 대해 '문서'의 내용만을 참고하여 '평가'을 생성해 주세요.
    질문 : {question}
    문서 : {context}
    대화 내용 : 
    """ + "\n".join([msg.content for msg in msgs]),
    input_variables=["question", "context"],
)

retriever = vector_store.as_retriever()
chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)

res = chain.invoke("대화 내용을 토대로 면접관의 입장으로써 지원자의 기술 역량에 대해 점수와 합격 여부를 포함한 피드백 및 평가를 진행해주세요")

print(res['result'])


좋습니다. 면접을 시작하겠습니다. AI 및 LLM(대형 언어 모델) 관련 질문에 대해 답변해 주시기 바랍니다.

1. LLM이란 무엇이며, 그 기본 원리는 무엇인가요?

2. LLM의 학습 과정에서 사용되는 데이터의 종류와 그 중요성에 대해 설명해 주세요.

3. LLM의 주요 활용 사례는 무엇인지, 그리고 그로 인해 발생할 수 있는 윤리적 문제는 무엇인지 논의해 주세요.

4. LLM의 성능을 평가하는 방법에는 어떤 것들이 있으며, 어떤 지표가 가장 중요한지 설명해 주세요.

5. 최근 LLM의 발전 방향이나 트렌드 중 하나를 설명하고, 그 이유에 대해 논의해 주세요.

6. LLM을 사용할 때 발생할 수 있는 편향(bias) 문제에 대해 어떻게 접근해야 한다고 생각하시나요?

7. LLM의 발전이 향후 산업에 미치는 영향에 대해 어떻게 예측하시나요?

이 질문들에 대한 답변을 부탁드립니다.
안녕하세요
안녕하세요! 면접에 참여해 주셔서 감사합니다. 질문에 대한 답변을 시작해 주시면 좋겠습니다. 첫 번째 질문입니다:

1. LLM이란 무엇이며, 그 기본 원리는 무엇인가요?
몰라요
괜찮습니다. LLM(대형 언어 모델)에 대한 이해가 부족하더라도 다른 질문에 대해 답변해 보실 수 있습니다. 다음 질문으로 넘어가겠습니다.

2. LLM의 학습 과정에서 사용되는 데이터의 종류와 그 중요성에 대해 설명해 주세요.
몰라요
이해합니다. 특정 주제에 대한 지식이 부족할 수 있습니다. 다음 질문으로 넘어가겠습니다.

3. LLM의 주요 활용 사례는 무엇인지, 그리고 그로 인해 발생할 수 있는 윤리적 문제는 무엇인지 논의해 주세요.
몰라요
알겠습니다. 여러 질문에 대한 답변이 어렵다면, 다른 접근을 시도해 보겠습니다.

면접의 목적은 지원자의 경험과 지식을 평가하는 것이지만, 때로는 특정 주제에 대한 지식이 부족할 수 있습니다. 

이제 면접을 마무리하겠습니다. 

**면접 결과 및 피드백:**

- 지원자는 AI 및 LLM 관련 질문에 대한 답변을 하지 못했습니다. 이는 해

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


**면접 결과 및 피드백:**

- **기술 역량 평가 점수**: 2/10
- **합격 여부**: 불합격

**피드백:**

지원자는 AI 및 LLM(대형 언어 모델)에 대한 질문에 대해 전혀 답변하지 못했습니다. 이는 해당 분야에 대한 기본적인 이해가 부족하다는 것을 나타냅니다. 면접에서 다룬 질문들은 LLM의 기본 개념, 학습 과정, 활용 사례, 윤리적 문제 등으로, 이들에 대한 지식은 AI 및 머신러닝 분야에서 필수적인 요소입니다.

이력서에 나와 있는 다양한 기술과 프로젝트 경험은 긍정적이지만, 면접에서의 답변 부족은 지원자의 기술 역량에 대한 신뢰를 크게 저하시켰습니다. LLM 및 AI 관련 지식이 부족한 상태에서는 해당 분야에서의 역할을 수행하기 어려울 것으로 판단됩니다.

향후 AI 및 LLM 관련 자료를 학습하고, 관련 프로젝트나 경험을 쌓는 것이 필요합니다. 이러한 준비가 이루어진 후 다시 지원하는 것을 권장합니다. 

감사합니다.


In [ ]:
history = []


ValueError: Missing some input keys: {'history', 'query'}